# Create Book-level Representation (ETL, Hyperparam, Architecture Search)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import itertools
import random
import fasttext as ft
import pickle
import optuna
from sklearn.model_selection import train_test_split

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from model import *

In [4]:
np.random.seed(1)
tf.random.set_seed(1)

## Load Data

### User History

In [5]:
his_pd = pd.read_csv('data/tosho_user_history.csv.bz2', sep='\t', compression='bz2')
his_pd.shape

(1400, 2)

In [6]:
his_pd.head()

,user_id,book_id
0,1,3901637
1,1,3901633
2,1,3901632
3,1,3901630
4,1,3901627


### Book Details

In [7]:
text_pd = pd.read_csv('data/tosho_processed_clean.csv.bz2', sep='\t', compression='bz2')
text_pd.shape

(938, 3)

In [8]:
text_pd.head()

,id,title,description_token
0,3901637,土のコレクション,土 なんて どれ も 同じ じゃ の 茶色 や の 土 しか 見 こと ない よ だれ も ...
1,3901633,わたしの,幼い 子ども たち が これ は 誰 の 物 と 意識 する よう に なる の は 2 歳...
2,3901632,なーらんだ,- ち ゅんちゅんとりさん ぶ ー ぶ ー くるま バラバラ に 散らばっ て い もの が...
3,3901630,つるのおんがえし,昔 ある ところ に 心 の 優しい 若者 が い ある 日 若者 は 罠 に かかっ て ...
4,3901627,ブレーメンのおんがくたい,ある ところ に 年 とっ ロバ が い 年 を とっ て 重い 荷物 を 運べ なっ ので...


In [9]:
text_dict = dict(zip(text_pd.id, text_pd.description_token))

## Data Augumentation

In [10]:
all_text_list = []
for user_id in his_pd['user_id'].unique():
    book_list = his_pd[his_pd.user_id == user_id]['book_id'].tolist()
    
    user_text_list = []
    for perm in itertools.permutations(book_list, 5):
        text_list = []
        for text_id in list(perm)[:4]:
            text_list.append(text_dict[text_id])
        text_list.append(list(perm)[-1])
        
        user_text_list.append(text_list)
    all_text_list.append(user_text_list)

## Convert user history into a series of text

Reshape all text_list to:  
`rows = num_user * ₅P₅`  
`columns = 5 books`

In [12]:
text_hist_pd = pd.DataFrame(np.reshape(np.array(all_text_list), \
                                       (int(len(all_text_list)*len(all_text_list[0])), 5)), \
                            columns=['book1', 'book2', 'book3', 'book4', 'label'])
text_hist_pd.shape

(33600, 5)

In [13]:
text_hist_pd.sample(3, random_state=1)

,book1,book2,book3,book4,label
17402,深い 霧 の たちこめる 漆 の 海 姿 なき 幽霊 船長 ゴースト キャプテン が 舵 を...,アイスダンス ペア 樹里 と 恵 は 初 出場 し 界 選手権 大会 で 大 喝采 を 浴び...,そろそろ 考え ちゃ 悩め ブタ シッタカブッタ は 人生 の 見え 方 を ていねい に ...,なぞなぞ 大好き の 妖怪 たち が 読者 に おもしろい なぞなぞ で いどみ 人気 の ...,1700233
9348,宗教 と は 国 と は 人間 へ の 深い まなざし が 捉え 青春 小説 の 新 境地 ...,わたし じゃ よ わ ー ん わ ー ん 学校 から の 帰り道 大きな 声 て い 清香 ...,担任 に 納得 の いか 絵 を 描か さ れ ユク は 傷つく 憧れ の 少女 に 励まさ...,ひな まつり に 愛花 の 家 で は お ばあちゃん だけ が つくれる ごちそう 肉まん...,1902265
32139,夜 の 星空 を 見 げた とき に 学校 で 習っ 星座 が 星座 の 名前 の 通り に...,界 の カエル と 日本 の カエル 200 種類 を カラー 写真 で 紹介 分布 や す...,現在 もっとも 謎 の 多い チョウ モンシロチョウ 最新 の 撮影 技術 と 研究 成果 ...,ほ 類 は虫類 ペット 家 ちく アニマル ウォッチング 絶滅 危機 動物 とじこみ 付録 ...,1400121


## Save Data Set

In [15]:
text_hist_pd.to_csv('data/text_hist_perm.csv.bz', sep='\t', compression='bz2', index=False)

## Sample 10% for Hyperparam Tuning

In [17]:
text_hist_sample_pd = text_hist_pd.sample(frac=.1, random_state=1)
text_hist_sample_pd.shape

(3360, 5)

## Split into Train and Val

In [18]:
train_texts, val_texts, y_train, y_val = train_test_split(text_hist_sample_pd.drop(['label'], axis=1),
                                                          text_hist_sample_pd[['label']].values,
                                                          test_size = .2,
                                                          random_state=1234)

## Tokenize

In [19]:
# Limit on the number of word features to the top 20K features.
TOP_K = 20000

# Most claims doc are within 10K word length
MAX_SEQUENCE_LENGTH = 500

def sequence_vectorize(train_texts, y_train, val_texts, y_val):
    # Create vocabulary with training texts.
    tokenizer = Tokenizer(num_words=TOP_K)
    tokenizer.fit_on_texts(train_texts['book1'])

    # Vectorize training and validation texts.
    # Fix sequence length to max value. Sequences shorter than the length are
    # padded in the beginning and sequences longer are truncated
    # at the beginning.
    x_trn_1 = pad_sequences(tokenizer.texts_to_sequences(train_texts['book1']), maxlen=MAX_SEQUENCE_LENGTH)
    x_trn_2 = pad_sequences(tokenizer.texts_to_sequences(train_texts['book2']), maxlen=MAX_SEQUENCE_LENGTH)
    x_trn_3 = pad_sequences(tokenizer.texts_to_sequences(train_texts['book3']), maxlen=MAX_SEQUENCE_LENGTH)
    x_trn_4 = pad_sequences(tokenizer.texts_to_sequences(train_texts['book4']), maxlen=MAX_SEQUENCE_LENGTH)
    x_val_1 = pad_sequences(tokenizer.texts_to_sequences(val_texts['book1']), maxlen=MAX_SEQUENCE_LENGTH)
    x_val_2 = pad_sequences(tokenizer.texts_to_sequences(val_texts['book2']), maxlen=MAX_SEQUENCE_LENGTH)
    x_val_3 = pad_sequences(tokenizer.texts_to_sequences(val_texts['book3']), maxlen=MAX_SEQUENCE_LENGTH)
    x_val_4 = pad_sequences(tokenizer.texts_to_sequences(val_texts['book4']), maxlen=MAX_SEQUENCE_LENGTH)
    
    return x_trn_1, x_trn_2, x_trn_3, x_trn_4, x_val_1, x_val_2, x_val_3, x_val_4, tokenizer.word_index, tokenizer

In [20]:
x_trn_1, x_trn_2, x_trn_3, x_trn_4, x_val_1, x_val_2, x_val_3, x_val_4, word_index, tokenizer = sequence_vectorize(train_texts, y_train, val_texts, y_val)

In [21]:
# number of vocabulary in the corpus
len(word_index)

12754

### Load FastText (Japanese)

In [22]:
ja_vecs = ft.load_model('model/cc.ja.300.bin')

In [23]:
ja_vecs.get_word_vector('あ').shape

(300,)

In [24]:
EMBEDDING_DIM = 300

## Use mean of FastText vectors as label

In [25]:
all_vocab_list = ja_vecs.get_words(on_unicode_error='replace')

def mean_embedding_vectorizer_transform(y, vecs=ja_vecs):
    return np.array([
        np.mean([vecs.get_word_vector(w) for w in words if w in all_vocab_list]
                or [np.zeros(EMBEDDING_DIM)], axis=0)
        for words in y
    ])

In [21]:
label_dict = {}
for book_id in text_pd.id.unique().tolist():
    label_dict[book_id] = text_dict[book_id]
    
unique_text = np.array([i for i in label_dict.values()])
unique_text_mean = mean_embedding_vectorizer_transform(unique_text, ja_vecs)

In [ ]:
label_vec_dict = dict(zip(text_pd.id.unique().tolist(), unique_text_mean))

In [42]:
np.save('data/label_vec_dict.npy', label_vec_dict)

In [27]:
label_vec_dict = np.load('data/label_vec_dict.npy', allow_pickle=True).tolist()

In [40]:
y_train_mean = []
for book_id in y_train:
    y_train_mean.append(label_vec_dict[int(book_id[0])])

y_train_mean = np.array(y_train_mean)

In [41]:
np.save('data/y_train_mean_sample.npy', y_train_mean)

In [20]:
y_train_mean = np.load('data/y_train_mean_sample.npy')

In [42]:
y_val_mean = []
for book_id in y_val:
    y_val_mean.append(label_vec_dict[int(book_id[0])])

y_val_mean = np.array(y_val_mean)

In [43]:
np.save('data/y_val_mean_sample.npy', y_val_mean)

In [21]:
y_val_mean = np.load('data/y_val_mean_sample.npy')

In [44]:
y_train.shape, y_train_mean.shape, y_val.shape, y_val_mean.shape

((2688, 1), (2688, 300), (672, 1), (672, 300))

## Model

### Embedding layer

In [45]:
embedding_matrix = np.zeros((TOP_K+1, EMBEDDING_DIM))
for word, i in word_index.items():
    if i < TOP_K:
        try:
            embedding_vector = ja_vecs.get_word_vector(word)
        except:
            embedding_vector = np.zeros(EMBEDDING_DIM)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [46]:
max_sequence = x_trn_1.shape[1] if MAX_SEQUENCE_LENGTH > x_trn_1.shape[1] else MAX_SEQUENCE_LENGTH

### Optuna for Hyperparameter Tuning

In [73]:
def objective(trial):
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    rnn_units = trial.suggest_int("rnn_unit", 150, 200)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 3e-2)
    share_gru_weights_on_book = trial.suggest_categorical('share_gru_weights_on_book', [True, False])
    use_attention_on_user = trial.suggest_categorical('use_attention_on_user', [True, False])
    use_batch_norm = trial.suggest_categorical('use_batch_norm', [True, False])
    is_embedding_trainable = trial.suggest_categorical('is_embedding_trainable', [True, False])
    final_activation = trial.suggest_categorical('final_activation', ['tanh', 'relu'])
    
    model = gru_model(
                     embedding_dim=EMBEDDING_DIM,
                     dropout_rate=dropout_rate,
                     rnn_unit=rnn_units,
                     input_shape=(max_sequence,),
                     num_features=TOP_K+1,
                     share_gru_weights_on_book=share_gru_weights_on_book,
                     use_attention_on_book=False,
                     use_attention_on_user=use_attention_on_user,
                     use_batch_norm=use_batch_norm,
                     is_embedding_trainable=is_embedding_trainable,
                     final_activation=final_activation,
                     embedding_matrix=embedding_matrix)

    # Compile model
    loss = 'cosine_similarity'
    optimizer = Adam(lr=learning_rate)
    model.compile(optimizer=optimizer, loss=loss, metrics=['cosine_similarity'])

    history = model.fit(
        [x_trn_1, x_trn_2, x_trn_3, x_trn_4],
        y_train_mean,
        epochs=1,
        validation_data=([x_val_1, x_val_2, x_val_3, x_val_4], y_val_mean),
        batch_size=256)
    
    return -np.amax(history.history['val_cosine_similarity'])

In [74]:
study = optuna.create_study()
study.optimize(objective, n_trials=100)

Train on 2688 samples, validate on 672 samples
2688/2688 [==============================] - 91s 34ms/sample - loss: -0.3388 - cosine_similarity: 0.3388 - val_loss: -0.5734 - val_cosine_similarity: 0.5734


[I 2019-10-04 18:01:39,102] Finished trial#0 resulted in value: -0.5733716487884521. Current best value is -0.5733716487884521 with parameters: {'dropout_rate': 0.24448834157804056, 'rnn_unit': 193, 'learning_rate': 0.00342548949866228, 'share_gru_weights_on_book': False, 'use_attention_on_user': False, 'use_batch_norm': True, 'is_embedding_trainable': False, 'final_activation': 'relu'}.


Train on 2688 samples, validate on 672 samples
2688/2688 [==============================] - 84s 31ms/sample - loss: -0.0598 - cosine_similarity: 0.0598 - val_loss: -0.4635 - val_cosine_similarity: 0.4635


[I 2019-10-04 18:03:21,204] Finished trial#1 resulted in value: -0.4634664058685303. Current best value is -0.5733716487884521 with parameters: {'dropout_rate': 0.24448834157804056, 'rnn_unit': 193, 'learning_rate': 0.00342548949866228, 'share_gru_weights_on_book': False, 'use_attention_on_user': False, 'use_batch_norm': True, 'is_embedding_trainable': False, 'final_activation': 'relu'}.


Train on 2688 samples, validate on 672 samples
2688/2688 [==============================] - 81s 30ms/sample - loss: -0.6129 - cosine_similarity: 0.6129 - val_loss: -0.8803 - val_cosine_similarity: 0.8803


...

Train on 2688 samples, validate on 672 samples
2688/2688 [==============================] - 84s 31ms/sample - loss: -0.7509 - cosine_similarity: 0.7509 - val_loss: -0.9175 - val_cosine_similarity: 0.9175


[I 2019-10-04 20:45:06,453] Finished trial#76 resulted in value: -0.9174818992614746. Current best value is -0.9247658848762512 with parameters: {'dropout_rate': 0.22677968569252382, 'rnn_unit': 196, 'learning_rate': 0.002635300602163673, 'share_gru_weights_on_book': True, 'use_attention_on_user': True, 'use_batch_norm': False, 'is_embedding_trainable': False, 'final_activation': 'tanh'}.


Train on 2688 samples, validate on 672 samples
2688/2688 [==============================] - 85s 32ms/sample - loss: -0.5143 - cosine_similarity: 0.5143


Exception ignored in: <function _EagerDefinedFunctionDeleter.__del__ at 0x7ff49c0159d8>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/eager/function.py", line 353, in __del__
    def __del__(self):
KeyboardInterrupt: 


KeyboardInterrupt: 

In [75]:
study.best_params

{'dropout_rate': 0.22677968569252382,
 'rnn_unit': 196,
 'learning_rate': 0.002635300602163673,
 'share_gru_weights_on_book': True,
 'use_attention_on_user': True,
 'use_batch_norm': False,
 'is_embedding_trainable': False,
 'final_activation': 'tanh'}

In [76]:
study.best_value

-0.9247658848762512

In [77]:
trial_df = study.trials_dataframe()

In [80]:
trial_df.sort_values(by='value')

number                state     value             datetime_start  \
                                                                      
32     32  TrialState.COMPLETE -0.924766 2019-10-04 18:59:16.007356   
71     71  TrialState.COMPLETE -0.924550 2019-10-04 20:29:09.207612   
61     61  TrialState.COMPLETE -0.924193 2019-10-04 20:04:13.050263   
62     62  TrialState.COMPLETE -0.922626 2019-10-04 20:06:40.165160   
31     31  TrialState.COMPLETE -0.922443 2019-10-04 18:57:12.387212   
30     30  TrialState.COMPLETE -0.921897 2019-10-04 18:55:10.700393   
55     55  TrialState.COMPLETE -0.921771 2019-10-04 19:49:48.922546   
49     49  TrialState.COMPLETE -0.921191 2019-10-04 19:36:09.681700   
60     60  TrialState.COMPLETE -0.920868 2019-10-04 20:01:46.034102   
26     26  TrialState.COMPLETE -0.920525 2019-10-04 18:47:08.907868   
47     47  TrialState.COMPLETE -0.919960 2019-10-04 19:31:37.521895   
42     42  TrialState.COMPLETE -0.919700 2019-10-04 19:20:31.919492   
51     51  TrialState.COMPLETE -0.919116 2019-10-04 19:40:41.306439   
64     64  TrialState.COMPLETE -0.918989 2019-10-04 20:11:32.027359   
34     34  TrialState.COMPLETE -0.918481 2019-10-04 19:03:19.296298   
53     53  TrialState.COMPLETE -0.918147 2019-10-04 19:45:12.406698   
72     72  TrialState.COMPLETE -0.918139 2019-10-04 20:31:37.843825   
75     75  TrialState.COMPLETE -0.917877 2019-10-04 20:39:35.132270   
68     68  TrialState.COMPLETE -0.917664 2019-10-04 20:21:31.851986   
73     73  TrialState.COMPLETE -0.917567 2019-10-04 20:34:20.553956   
76     76  TrialState.COMPLETE -0.917482 2019-10-04 20:42:21.101366   
56     56  TrialState.COMPLETE -0.916324 2019-10-04 19:52:18.551022   
19     19  TrialState.COMPLETE -0.915411 2019-10-04 18:33:47.262500   
57     57  TrialState.COMPLETE -0.915052 2019-10-04 19:54:41.057274   
41     41  TrialState.COMPLETE -0.914749 2019-10-04 19:18:18.936315   
67     67  TrialState.COMPLETE -0.914524 2019-10-04 20:18:59.924681   
69     69  TrialState.COMPLETE -0.914201 2019-10-04 20:24:09.585430   
24     24  TrialState.COMPLETE -0.914138 2019-10-04 18:43:13.658770   
52     52  TrialState.COMPLETE -0.913003 2019-10-04 19:42:57.770978   
15     15  TrialState.COMPLETE -0.912610 2019-10-04 18:26:21.511185   
..    ...                  ...       ...                        ...   
74     74  TrialState.COMPLETE -0.840309 2019-10-04 20:36:59.557820   
54     54  TrialState.COMPLETE -0.837474 2019-10-04 19:47:33.868911   
28     28  TrialState.COMPLETE -0.817527 2019-10-04 18:51:06.845261   
66     66  TrialState.COMPLETE -0.759489 2019-10-04 20:16:29.602214   
22     22  TrialState.COMPLETE -0.725398 2019-10-04 18:39:21.783321   
50     50  TrialState.COMPLETE -0.708767 2019-10-04 19:38:24.497070   
10     10  TrialState.COMPLETE -0.706545 2019-10-04 18:17:16.896645   
46     46  TrialState.COMPLETE -0.678827 2019-10-04 19:29:23.292663   
58     58  TrialState.COMPLETE -0.676540 2019-10-04 19:57:02.612780   
13     13  TrialState.COMPLETE -0.673093 2019-10-04 18:22:42.803367   
37     37  TrialState.COMPLETE -0.671255 2019-10-04 19:09:50.774701   
18     18  TrialState.COMPLETE -0.608246 2019-10-04 18:31:55.351060   
70     70  TrialState.COMPLETE -0.605611 2019-10-04 20:26:41.035852   
17     17  TrialState.COMPLETE -0.586819 2019-10-04 18:30:00.098854   
59     59  TrialState.COMPLETE -0.586530 2019-10-04 19:59:25.869602   
27     27  TrialState.COMPLETE -0.581141 2019-10-04 18:49:08.922742   
0       0  TrialState.COMPLETE -0.573372 2019-10-04 17:59:49.424741   
35     35  TrialState.COMPLETE -0.560075 2019-10-04 19:05:26.286751   
40     40  TrialState.COMPLETE -0.533759 2019-10-04 19:16:07.894287   
3       3  TrialState.COMPLETE -0.527054 2019-10-04 18:05:01.261276   
16     16  TrialState.COMPLETE -0.516097 2019-10-04 18:28:08.640633   
5       5  TrialState.COMPLETE -0.508087 2019-10-04 18:08:31.717390   
4       4  TrialState.COMPLETE -0.503993 2019-10-04 18:06:47.214501   
25     25  TrialState.COMPL